In [3]:
#Library Imports
import numpy as np
import pandas as pd
import math
import os
import matplotlib.pyplot as plt

from sklearn.metrics import mean_absolute_error

#######딥러닝 라이브러리##########
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Reshape, GRU, RNN
import sklearn
tf.keras.backend.set_floatx('float64')

In [5]:
train=pd.read_csv('Train_13.csv',encoding='cp949' )
test=pd.read_csv('Test_13.csv',encoding='cp949')

In [6]:
#train.shape 122400 X 10
#60개의 건물 X 85일 24시간 =122400
train

,date,time,전력사용량(kWh),기온(°C),풍속(m/s),습도(%),강수량(mm),일조(hr),비전기냉방설비운영,태양광보유
0,2020-06-01,0,640.872,19.0,0.7,80,0.0,0.0,1,0
1,2020-06-01,1,626.778,18.6,0.5,81,0.0,0.0,1,0
2,2020-06-01,2,623.538,19.0,0.7,79,0.0,0.0,1,0
3,2020-06-01,3,653.022,18.7,0.6,79,0.0,0.0,1,0
4,2020-06-01,4,692.874,18.3,0.7,83,0.0,0.0,1,0
...,...,...,...,...,...,...,...,...,...,...
1963,2020-08-21,19,1134.972,28.4,0.2,72,0.0,0.0,1,0
1964,2020-08-21,20,1062.882,27.0,0.7,76,0.0,0.0,1,0
1965,2020-08-21,21,1009.098,25.5,0.3,83,0.0,0.0,1,0
1966,2020-08-21,22,965.034,25.8,0.6,83,0.0,0.0,1,0


In [7]:
#test.shape 10080 X 9
#60개의 건물 X 7일 24시간 =10080
test

,date,time,전력사용량(kWh),기온(°C),풍속(m/s),습도(%),강수량(mm),일조(hr),비전기냉방설비운영,태양광보유
0,2020-08-22,0,941.544,25.8,0.4,84,0,0.0,1,0
1,2020-08-22,1,935.874,25.6,0.2,86,0,0.0,1,0
2,2020-08-22,2,911.574,25.2,1.4,85,0,0.0,1,0
3,2020-08-22,3,928.260,25.2,0.9,82,0,0.0,1,0
4,2020-08-22,4,931.014,24.6,1.6,86,0,0.0,1,0
...,...,...,...,...,...,...,...,...,...,...
67,2020-08-24,19,1075.194,28.0,0.4,70,0,0.4,1,0
68,2020-08-24,20,1042.794,27.2,0.2,75,0,0.0,1,0
69,2020-08-24,21,1002.618,26.4,0.7,76,0,0.0,1,0
70,2020-08-24,22,984.312,26.3,0.3,78,0,0.0,1,0


 # 전처리

In [8]:
train[['비전기냉방설비운영','태양광보유']]
ice={}
hot={}
count=0
for i in range(0, len(train), len(train)//60):
    count +=1
    ice[count]=train.loc[i,'비전기냉방설비운영']
    hot[count]=train.loc[i,'태양광보유']
    

In [9]:
train.interpolate(method='values')

,date,time,전력사용량(kWh),기온(°C),풍속(m/s),습도(%),강수량(mm),일조(hr),비전기냉방설비운영,태양광보유
0,2020-06-01,0,640.872,19.0,0.7,80,0.0,0.0,1,0
1,2020-06-01,1,626.778,18.6,0.5,81,0.0,0.0,1,0
2,2020-06-01,2,623.538,19.0,0.7,79,0.0,0.0,1,0
3,2020-06-01,3,653.022,18.7,0.6,79,0.0,0.0,1,0
4,2020-06-01,4,692.874,18.3,0.7,83,0.0,0.0,1,0
...,...,...,...,...,...,...,...,...,...,...
1963,2020-08-21,19,1134.972,28.4,0.2,72,0.0,0.0,1,0
1964,2020-08-21,20,1062.882,27.0,0.7,76,0.0,0.0,1,0
1965,2020-08-21,21,1009.098,25.5,0.3,83,0.0,0.0,1,0
1966,2020-08-21,22,965.034,25.8,0.6,83,0.0,0.0,1,0


In [12]:
def weekday(x):
    return pd.to_datetime(x[:10]).weekday()
train['weekday']=train['date'].apply(lambda x :weekday(x))
test['weekday']=test['date'].apply(lambda x :weekday(x))

In [13]:
train

,date,time,전력사용량(kWh),기온(°C),풍속(m/s),습도(%),강수량(mm),일조(hr),비전기냉방설비운영,태양광보유,weekday
0,2020-06-01,0,640.872,19.0,0.7,80,0.0,0.0,1,0,0
1,2020-06-01,1,626.778,18.6,0.5,81,0.0,0.0,1,0,0
2,2020-06-01,2,623.538,19.0,0.7,79,0.0,0.0,1,0,0
3,2020-06-01,3,653.022,18.7,0.6,79,0.0,0.0,1,0,0
4,2020-06-01,4,692.874,18.3,0.7,83,0.0,0.0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...
1963,2020-08-21,19,1134.972,28.4,0.2,72,0.0,0.0,1,0,4
1964,2020-08-21,20,1062.882,27.0,0.7,76,0.0,0.0,1,0,4
1965,2020-08-21,21,1009.098,25.5,0.3,83,0.0,0.0,1,0,4
1966,2020-08-21,22,965.034,25.8,0.6,83,0.0,0.0,1,0,4


In [14]:
train_x=train.drop('전력사용량(kWh)', axis=1)
test_y=train[['전력사용량(kWh)']]

In [16]:
train_x.drop('date', axis=1, inplace=True)

# **딥러닝 모델(전력사용량만 변수로 사용)**

In [17]:
##############전력사용량(kWh) 정규화####################################

mini=train_x.iloc[:,2].min()
size=train_x.iloc[:,2].max()-train_x.iloc[:,2].min()
train_x.iloc[:,2]=(train_x.iloc[:,2]-mini)/size

In [18]:
mini=test_y.iloc[:,0].min()
size=test_y.iloc[:,0].max()-test_y.iloc[:,0].min()
test_y.iloc[:,0]=(test_y.iloc[:,0]-mini)/size

C:\Users\Admin\anaconda3\lib\site-packages\pandas\core\indexing.py:1783: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item_labels[indexer[info_axis]]] = value


In [19]:
train_x

,time,기온(°C),풍속(m/s),습도(%),강수량(mm),일조(hr),비전기냉방설비운영,태양광보유,weekday
0,0,19.0,0.120690,80,0.0,0.0,1,0,0
1,1,18.6,0.086207,81,0.0,0.0,1,0,0
2,2,19.0,0.120690,79,0.0,0.0,1,0,0
3,3,18.7,0.103448,79,0.0,0.0,1,0,0
4,4,18.3,0.120690,83,0.0,0.0,1,0,0
...,...,...,...,...,...,...,...,...,...
1963,19,28.4,0.034483,72,0.0,0.0,1,0,4
1964,20,27.0,0.120690,76,0.0,0.0,1,0,4
1965,21,25.5,0.051724,83,0.0,0.0,1,0,4
1966,22,25.8,0.103448,83,0.0,0.0,1,0,4


In [20]:
test_y

,전력사용량(kWh)
0,0.013650
1,0.002551
2,0.000000
3,0.023217
4,0.054599
...,...
1963,0.402730
1964,0.345962
1965,0.303610
1966,0.268912


In [21]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(train_x,test_y, test_size=0.2,random_state=100)

X_val, X_test, y_val, y_test = train_test_split(train_x,test_y, test_size=0.5, random_state=100)

print(X_train.shape)
print(X_val.shape)
print('======'*2)
print(y_train.shape)
print(y_val.shape)

(1574, 9)
(984, 9)
(1574, 1)
(984, 1)


In [22]:
from tensorflow import keras
from tensorflow.keras import layers

# 위의 코드를 한줄로 표현하기 model.add(Dense(units=1, input_dim=2, activation='sigmoid'))
# model.add(Dense(128, input_shape=x_train[0].shape, activation='relu'))
#model.add(Dense(128, input_dim=x_teain.shape[1]))from tensorflow.keras.layers import Dense, Dropout

In [32]:
model = keras.Sequential()
model.add(Dense(128, input_shape= X_train.shape, activation='relu'))
model.add(layers.Activation('relu'))
# model.add(kernel_regularizer = keras.regularizers.L2(0.1))

model.add(layers.Dense(units=32))
model.add(layers.Activation('relu'))

model.add(layers.Dense(units=1))
model.add(layers.Activation('linear'))
model.summary() # summary로 layer의 형태 파악

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_5 (Dense)              (None, 1574, 128)         1280      
_________________________________________________________________
activation_5 (Activation)    (None, 1574, 128)         0         
_________________________________________________________________
dense_6 (Dense)              (None, 1574, 32)          4128      
_________________________________________________________________
activation_6 (Activation)    (None, 1574, 32)          0         
_________________________________________________________________
dense_7 (Dense)              (None, 1574, 1)           33        
_________________________________________________________________
activation_7 (Activation)    (None, 1574, 1)           0         
Total params: 5,441
Trainable params: 5,441
Non-trainable params: 0
____________________________________________________

In [45]:
from keras.callbacks import ModelCheckpoint
adam= keras.optimizers.Adam(lr=0.1)    
model.compile(loss='mean_squared_error', optimizer='Adam', metrics=['mae'])

C:\Users\Admin\anaconda3\lib\site-packages\tensorflow\python\keras\optimizer_v2\optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


In [46]:
history= model.fit(X_train, y_train,
          batch_size=32,
          epochs=100,
          verbose=1, 
          validation_split= 0.2) 

Epoch 1/100
40/40 [==============================] - 1s 5ms/step - loss: 0.2877 - mae: 0.3991 - val_loss: 0.0183 - val_mae: 0.1125
Epoch 2/100
40/40 [==============================] - 0s 2ms/step - loss: 0.0134 - mae: 0.0907 - val_loss: 0.0118 - val_mae: 0.0843
Epoch 3/100
40/40 [==============================] - 0s 2ms/step - loss: 0.0079 - mae: 0.0679 - val_loss: 0.0099 - val_mae: 0.0722
Epoch 4/100
40/40 [==============================] - 0s 2ms/step - loss: 0.0076 - mae: 0.0663 - val_loss: 0.0107 - val_mae: 0.0784
Epoch 5/100
40/40 [==============================] - 0s 2ms/step - loss: 0.0096 - mae: 0.0754 - val_loss: 0.0121 - val_mae: 0.0838
Epoch 6/100
40/40 [==============================] - 0s 2ms/step - loss: 0.0086 - mae: 0.0712 - val_loss: 0.0115 - val_mae: 0.0766
Epoch 7/100
40/40 [==============================] - 0s 2ms/step - loss: 0.0076 - mae: 0.0658 - val_loss: 0.0214 - val_mae: 0.1145
Epoch 8/100
40/40 [==============================] - 0s 2ms/step - loss: 0.0091 - m

40/40 [==============================] - 0s 2ms/step - loss: 0.0096 - mae: 0.0765 - val_loss: 0.0125 - val_mae: 0.0819
Epoch 56/100
40/40 [==============================] - 0s 2ms/step - loss: 0.0127 - mae: 0.0874 - val_loss: 0.0171 - val_mae: 0.0926
Epoch 57/100
40/40 [==============================] - 0s 2ms/step - loss: 0.0107 - mae: 0.0814 - val_loss: 0.0255 - val_mae: 0.1317
Epoch 58/100
40/40 [==============================] - 0s 2ms/step - loss: 0.0107 - mae: 0.0805 - val_loss: 0.0130 - val_mae: 0.0845
Epoch 59/100
40/40 [==============================] - 0s 2ms/step - loss: 0.0089 - mae: 0.0735 - val_loss: 0.0096 - val_mae: 0.0711
Epoch 60/100
40/40 [==============================] - 0s 2ms/step - loss: 0.0091 - mae: 0.0741 - val_loss: 0.0153 - val_mae: 0.0877
Epoch 61/100
40/40 [==============================] - 0s 2ms/step - loss: 0.0112 - mae: 0.0838 - val_loss: 0.0307 - val_mae: 0.1356
Epoch 62/100
40/40 [==============================] - 0s 2ms/step - loss: 0.0170 - mae: 0

In [47]:
train_result = model.evaluate(X_train, y_train)
test_result = model.evaluate(X_test, y_test)

print("train_result:", train_result)
print("test_result", test_result )

31/31 [==============================] - 0s 899us/step - loss: 0.0088 - mae: 0.0714
train_result: [0.007873726750722834, 0.06689042586033832]
test_result [0.008755509771891242, 0.0713654698507097]


In [48]:
prediction= model.predict(X_test)

In [49]:
prediction

array([[ 7.42106477e-01],
       [ 3.27860998e-01],
       [ 3.24448994e-01],
       [ 3.52299337e-01],
       [ 2.08473692e-01],
       [ 8.22450053e-01],
       [ 2.12363455e-01],
       [ 5.83365736e-01],
       [ 9.08006635e-01],
       [ 1.77060424e-01],
       [ 8.35121002e-02],
       [ 1.40890315e-01],
       [ 2.18362073e-01],
       [ 7.32986248e-01],
       [ 3.61877973e-01],
       [ 2.61408629e-01],
       [ 4.56114659e-02],
       [ 6.64574238e-01],
       [ 3.02774684e-01],
       [ 4.50586441e-01],
       [ 5.12942162e-01],
       [ 5.42507529e-02],
       [ 5.08606926e-01],
       [ 6.99160208e-01],
       [ 3.89489503e-01],
       [ 1.97974766e-01],
       [ 1.51937901e-01],
       [ 6.11222137e-02],
       [ 3.19859124e-01],
       [ 1.17118422e-01],
       [ 1.25892625e-01],
       [ 5.90186555e-02],
       [ 2.59474239e-01],
       [ 5.84001882e-01],
       [-3.11346772e-02],
       [ 1.86537542e-01],
       [ 5.77960249e-01],
       [ 6.73691633e-01],
       [ 1.4

In [51]:
def SMAPE(true, pred):
    return np.mean((np.abs(true-pred))*2/(np.abs(true) + np.abs(pred)))*100

print(SMAPE(y_test,prediction))

전력사용량(kWh)    30.544637
dtype: float64
